# AITA Classifier
This project uses datasets scraped from the subreddit *r/amitheasshole* to fine tune a BERT LLM for sentence classification to determine whether the person in the given example is or isn't an asshole.

The *r/amitheasshole* subreddit consists of posts, where people present situations that they have experienced where they are unable to determine if they are the person in the wrong or not. Then, people in the community respond with "YTA" or "NTA", standing for "You're the asshole" or "not the asshole", respectively. Following this label, people often give a brief description explaining how they reached their decision. 

Posts are upvoted or downvoted by other users, and for this application, the highest upvoted post that provided a "YTA"/"NTA" label was used as the true label. While this may not do a good job of estimating the actual moral situation, it is a good model of how people on the website might respond.

With the fine tuned BERT model, we were able to achieve a classification accuracy of 95%. 

In addition to this, another pretrained language model from HuggingFace was employed to calculate the semantic similarity between each context and a list of different senses that could explain the root cause of the problem in each situation. A rolling semantic similarity is employed, as the input sentences tend to be quite long while the class labels are quite short. In this method, each n-gram of the sentence is encoded, and their cosine similarities are computed relative to the each of the possible senses. Then, the sense with the highest average cosine similarity across all n-grams for the sentence is chosen as the reason, and this is returned to the user. 

# IMPORT LIBRARIES

In [1]:
!pip install transformers
!pip install sentence-transformers
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig, get_linear_schedule_with_warmup
from sentence_transformers import SentenceTransformer
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm, trange
import torch
from torch.optim import AdamW
from torch import device
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import os
import pandas as pd
import numpy as np
import re 
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import porter
import random

print("IMPORTED LIBRARIES SUCCESSFULLY")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.9 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=30f2b9cfdf696b45e143f85a8a02617473e6646c442226c0e98fcc9b164428c7
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
IMPORTED LIBRARIES SUCCESSFULLY


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [2]:
try:
  from google.colab import drive
  drive.mount('/content/drive')
  os.chdir('/content/drive/MyDrive/AITA/data')
  nta = pd.read_json("NTA_Dataset.jsonl", lines=True)
  yta = pd.read_json("YTA_Dataset.jsonl", lines=True)
  nta['label'] = 0
  yta['label'] = 1
except:
  print("Could not import drive from google colab")

Mounted at /content/drive


# DATA PROCESSING AND PRE-PROCESSING

In [3]:
def preprocess(data):
  # in: 2d table with prompts and resolution
  # out: tidied data
  # ends with 5 hashtags
  # remove capitalisation?
  # 
  data['completion'] = data['completion'].str.lower()
  data['completion'] = data['completion'].str.replace(r'#', '', regex=True)
  #data['completion'] = data['completion'].str.replace(r'[N,n,Y,y][T,t][A,a]', '', regex=True)
  # This currently replaces *all* occurences of "nta" and "yta". "Contact"->"Coct"
  data['completion'] = data['completion'].str.remove_prefix(r'[NnYy][Tt][Aa]', 1, regex=True)
  data['completion'] = data['completion'].str.replace(r'\s+', ' ', regex=True)
  return data

def correct_labels(data):
  data.loc[(data["completion"].str.lower()).str.startswith(' definitely nta'), 'label'] = 0
  return data

yta = correct_labels(yta)
data = pd.concat([nta,yta,yta])
data.drop(["completion"],axis=1,inplace=True)

In [4]:
max_len = 256
prompts = data.prompt.values
labes = data.label.values
prompt_ids = []
masks = []
batch_size = 16

# INITIALIZING TOKENIZER
We start here with a pretrained BERT model, and use these to encode the prompts, add special tokens necessary for classification, and truncate prompts to 256.
Additionally, we create masks to indicate to the model where padding has been inserted.

In [5]:
# create tokenizer
t = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = True)

for i in trange(len(prompts)):
  prompt = prompts[i]
  e = t.encode(prompt, add_special_tokens = True, truncation=True, max_length=max_len)
  prompt_ids.append(e)

prompt_ids = pad_sequences(prompt_ids, maxlen=max_len, padding="post")

for p in prompt_ids:
  masks.append([int(id > 0 ) for id in p])

100%|██████████| 10238/10238 [01:11<00:00, 142.42it/s]


# DATA PREPERATION
Here, we split the tokenized prompts and their masks into training and testing sets.
After this, we create a dataloader for fine-tuning the BERT model by converting this data into tensors, creating a tensor dataset, and creating a random sampler with the dataset. These are all passed as arguments into the DataLoader, which creates an iterable over the dataset for the model fine-tuning.

In [6]:
tr_in, te_in, tr_lab, te_lab = train_test_split(prompt_ids, labes, test_size = .2)
tr_masks, te_masks, _, _ = train_test_split(masks, labes, test_size=.2)

tr_in = torch.tensor(tr_in)
te_in = torch.tensor(te_in)
tr_lab = torch.tensor(tr_lab)
te_lab = torch.tensor(te_lab)
tr_masks = torch.tensor(tr_masks)
te_masks = torch.tensor(te_masks)

tr_d = TensorDataset(tr_in, tr_masks, tr_lab)
tr_s = RandomSampler(tr_d)
tr_dl = DataLoader(tr_d, sampler = tr_s, batch_size = batch_size)
te_d = TensorDataset(te_in, te_masks, te_lab)
te_s = RandomSampler(te_d)
te_dl = DataLoader(te_d, sampler = te_s, batch_size = batch_size)

# MODEL CREATION
Here, we import a pretrained BERT model for sequence classification for fine-tuning. Additionally, we establish our optimzer, AdamW.

In [7]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 2, output_attentions = False, output_hidden_states = False)
optimzier = AdamW(model.parameters(), lr=2e-5)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [8]:
try:
  device = torch.device("cuda")
  model.to(device)
except:
  device = torch.device("cpu")
  model.to(device)

# MODEL LOADING
Here, we load our fine-tuned BERT model that has been trained for 4 epochs with a lenear scheduler with warmup using the training dataloader. 

In [9]:
os.chdir('/content/drive/MyDrive/AITA/models')
chp = torch.load("pytorch_model2.bin")
model.load_state_dict(chp)

<All keys matched successfully>

# MODEL TESTING
This function "test" takes as input a prompt, and outputs a label generated by the model, either "YTA" or "NTA".

In [10]:
def test(sent):
  with torch.device("cuda") as device:
    predict_ids = []
    predict_mask = []
    predict_encoding = t.encode(sent, add_special_tokens = True, truncation=True, max_length=max_len)
    predict_ids.append(predict_encoding)
    predict_mask.append([int(id > 0 ) for id in predict_encoding])
    predict_ids = torch.tensor(predict_ids)
    predict_mask = torch.tensor(predict_mask)

    with torch.no_grad():
      output = model(predict_ids.to(device), token_type_ids = None, attention_mask = predict_mask)

    if np.argmax(output.logits.cpu()).flatten().item() == 1:
      return "YTA"
    else:
      return "NTA"

# DEMONSTRATION
This short demonstration randomly chooses 10 prompts and feeds them to the model to demonstrate the typical input, the predicted output class, and the actual output class.

In [11]:
for i in range(10):
  loc = random.randint(0, len(data.prompt.values)-1)
  print("Prompt: " +data.prompt.values[loc])
  if data.label.values[loc] == 0:
    lab = "NTA"
  else:
    lab = "YTA"
  print("Actual label: " +lab)
  print("Predicted label: " +test(data.prompt.values[loc]))
  print("\n")

Prompt: Before everyone judges me as an asshole, let me give a little backstory. 

My brother in law got me a pair of really awesome boots 3 Christmases ago. I loved them. However, he got me the wrong size. I literally could not fit my foot into them. I asked my husband to ask my brother in law if he could exchange the shoes for the right size...and it just never happened. This pair of shoes has been sitting in my garage for over 3 years now because my brother in law has said he just doesn't have the time or the receipt anymore to get the right size.

So, today, I sold them. Money has been really tight with both my husband and I laid off and not receiving unemployment/stimulus yet. I made a pretty profit too because they are high end shoes that have literally never been worn.

My husband is really angry at me for. Called me ungrateful and that I should be "ashamed of myself" for selling them. Even though now we will be able to pay the light bill and get a few groceries after selling th

# MODEL METRICS
Here, the testing accuracy, precision, recall, and F1 score are calculated.
The model demonstrates exeptional performance in all categories, demonstrating that the fine-tuning process worked extremely well. 

In [12]:
def b_metrics(preds, labels):
  preds = np.argmax(preds, axis = 1).flatten()
  labels = labels.flatten()
  tp = sum([preds == labels and preds == 1 for preds, labels in zip(preds, labels)])
  tn = sum([preds == labels and preds == 0 for preds, labels in zip(preds, labels)])
  fp = sum([preds != labels and preds == 1 for preds, labels in zip(preds, labels)])
  fn = sum([preds != labels and preds == 0 for preds, labels in zip(preds, labels)])
  b_accuracy = (tp + tn) / len(labels)
  b_precision = tp / (tp + fp) if (tp + fp) > 0 else 'nan'
  b_recall = tp / (tp + fn) if (tp + fn) > 0 else 'nan'
  b_specificity = tn / (tn + fp) if (tn + fp) > 0 else 'nan'
  return b_accuracy, b_precision, b_recall, b_specificity

model.eval()

val_acc = []
val_p = []
val_recall = []
val_spec = []

for batch in te_dl:
  with torch.device('cuda') as device:
    b = tuple(t.to(device) for t in batch)
    b_id = batch[0].to(device)
    b_m = batch[1].to(device)
    b_lab = batch[2].to(device)
    with torch.no_grad():
      eval_out = model(b_id, token_type_ids=None, attention_mask=b_m)
      logits = eval_out.logits.detach().cpu().numpy()
      label_ids = b_lab.to('cpu').numpy()
      b_acc, b_prec, b_recall, b_spec = b_metrics(logits, label_ids)
      val_acc.append(b_acc)
      if b_prec != 'nan': val_p.append(b_prec)
      if b_recall != 'nan': val_recall.append(b_recall)
      if b_spec != 'nan': val_spec.append(b_spec)


print('\t - Testing Accuracy: {:.4f}'.format(sum(val_acc)/len(val_acc)))
prec = sum(val_p)/len(val_p)
rec = sum(val_recall)/len(val_recall)
print('\t - Testing Precision: {:.4f}'.format(prec) if len(val_p)>0 else '\t - Validation Precision: NaN')
print('\t - Testing Recall: {:.4f}'.format(rec) if len(val_recall)>0 else '\t - Validation Recall: NaN')
print('\t - Testing F1: {:.4f}\n'.format((2*prec*rec)/(prec+rec)) if len(val_recall)>0 and len(val_p)>0 else '\t - Validation Specificity: NaN')

	 - Testing Accuracy: 0.9722
	 - Testing Precision: 0.9557
	 - Testing Recall: 0.9887
	 - Testing F1: 0.9719



# JUDGEMENT CASTING
In addition to the task of YTA/NTA classification, we wanted to be able to provide users with a justification or reason for the verdict. The way that this was implimented was with a pretrained Semantic Similarity sentence transformer, all-MiniLM0L6-V2.

The sentences are preprocessed to remove stopwords, stemmed with the Porter Stemmer, and lemmatized with wordnet lemmatizer. After this, word embeddings are generated using the sentence transofmrer.

Categories were chosen by us based on common themes throughout the posts; while they may not capture all possible situations, they are nuanced enough to provide some insite on the situation. 

These categories are embedded using the same model.
Semantic similarity is then calculated using cosine similarity using a method similar to that propsed by Shashavli et al. 

# TEXT PRE-PROCESSING

In [13]:
stops = stopwords.words("english")
lemmatizer = nltk.WordNetLemmatizer()
prompts = data.prompt.values

def process_prompt(sentence):
  sent = re.sub('[^\w\s]', '', sentence.lower().strip()).split()
  stemmer = porter.PorterStemmer()
  t_sent = [w for w in sent if w not in stops]
  t_sent = [stemmer.stem(w) for w in t_sent]
  t_sent = [lemmatizer.lemmatize(w) for w in t_sent]

  return " ".join(t_sent)

prompts_clean = [process_prompt(p) for p in prompts]
categories = ["irresponsiblity","entitlement","stupidity","bigotry","miscommunication","trauma","betrayal","dangerousness","obsession","pride","insecurity","rudeness"]

# TEXT ENCODING

In [14]:
m = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
embeddings = m.encode(prompts_clean[:20])
cats_emb = m.encode(categories)

# SIMILARITY METHOD
Because the prompts are so long and the categories are so short, calculating the semantic similarity between the whole sentence and individual category is not the best approach.
The method used to counter this is to use a context window that slides along the length of the prompt. The semantic similarity between the n-gram in the context window and the individual categoriies is calculated, and the average semantic similarity over all n-grams for the prompt is used to decide the final category chosen by the model.

In [15]:
def calc_sliding_cosine_similarity(sent, embs, w):
  sims = []
  for i in range(len(sent)-w):
    sims.append(cosine_similarity([m.encode(sent[i:i+w])], embs))
  sims = np.array(sims).T
  sims =np.array([a[0] for a in sims])
  sims_avg = [np.mean(sim) for sim in sims]
  closest = np.argmax(sims_avg)
  return closest

# BLAME ASSIGNMENT
Here we demonstrate the similarity model in action, paired with the classification technique designed earlier. For each prompt, a label is assigned by the fine-tuned BERT model. Following this, the semantic similarity is calculated, and blame is assigned to either you (YTA) or a third party (NTA).

In [16]:
for j in range(10):
  i = random.randint(0, len(prompts_clean)-1)
  sentence = prompts_clean[i]
  print(prompts[i])
  resp = test(prompts_clean[i])
  print("Response: " + resp)
  if resp == "NTA":
    reason = "The issue in this instance was their " +categories[calc_sliding_cosine_similarity(sentence, cats_emb, 15)]
  else:
    reason = "The issue in this instance was your" +categories[calc_sliding_cosine_similarity(sentence, cats_emb, 15)]
  print("Reason: " + reason)
  print("\n")
  print("**************************")
  print("\n")

Was checking out at the self-checkout. Realized I had two extra frozen meals that I did not need. Crazy time right now - just wanted to get out quickly so I set them on top of the candy rack in front of me.

Store manager confronts me. She says “are you planning on putting that back”? Like for fucks sake, am I back in school? Is this not what she has paid employees for? I said “I’m in a hurry, sorry” and bolted. ######
Response: NTA
Reason: The issue in this instance was their betrayal


**************************


I have a son [16M] and a daughter [15F]. In the same weekend, my son was broken up with by his girlfriend and my daughter was broken up with by her boyfriend. They were very upset. Since they were going through the same thing, they decided to be there for the other and spend more time together. They are spending all day together talking, crying, eating, watching movies, playing video games, etc. I've heard them say things like they don't need a relationship as they have eac

KeyboardInterrupt: ignored

# DEMO
Try your own prompt, and see what the classifier and judgement caster thinks!

In [22]:
custom_prompt = ... #ENTER YOUR PROMPT HERE
custom_prompt = "m extremely entitled and hurt my feelings"
cls = test(custom_prompt)
reason = categories[calc_sliding_cosine_similarity(custom_prompt, cats_emb, 15)]

if cls == "NTA":
  print("You are NTA!")
  print("The issue in this instance was their" +reason)
else:
  print("YTA!")
  print("The issue in this instance was your " +reason)

YTA!
The issue in this instance was your entitlement
